### Import Libraries

In [1]:
import pydot
import numpy as np
from IPython.display import SVG, display

from pydrake.all import Simulator, DiagramBuilder, AddMultibodyPlantSceneGraph,\
                        Parser, RigidTransform, MeshcatVisualizer, MeshcatVisualizerParams, \
                        ConstantVectorSource, ConstantValueSource, PiecewisePolynomial,\
                        AbstractValue, HalfSpace, CoulombFriction, LeafSystem, BasicVector, \
                        MultibodyPlant, InverseDynamicsController
import footstep_planner
import osc
import importlib
importlib.reload(osc)
importlib.reload(footstep_planner)
from osc import OperationalSpaceWalkingController, OscGains

/shared-libs/python3.8/py/lib/python3.8/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.17.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Start Meshcat

In [2]:
from pydrake.all import StartMeshcat

meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://c140ec4a-6224-46c8-b55a-d8a7a25ad3a5.deepnoteproject.com/7001/


### Add Humanoid Walker URDF as the Simulation Plant

In [3]:
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
X_WG = HalfSpace.MakePose(np.array([0,0, 1]), np.zeros(3,))
plant.RegisterCollisionGeometry(
    plant.world_body(), 
    X_WG, HalfSpace(), 
    "collision", 
    CoulombFriction(1.0, 1.0)
)
# Note that we parse into both the plant and the scene_graph here.
Parser(plant, scene_graph).AddModels("humanoid_walker.urdf")
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base"))
plant.Finalize()

# Add MeshcatVisualizer for visualization
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

### Use Double Pendulum URDF to calculate PID Control Input for the Arms

In [4]:
# Secondary MultibodyPlant for control
control_plant = MultibodyPlant(time_step=1e-4)
control_model = Parser(control_plant).AddModels("double_pendulum.urdf")[0]
control_plant.Finalize()

# Define PID gains and add the inverse dynamics controller
kp = [100] * control_plant.num_positions()
ki = [1] * control_plant.num_positions()
kd = [20] * control_plant.num_positions()
left_arm_controller = builder.AddSystem(InverseDynamicsController(control_plant, kp, ki, kd, False))
left_arm_controller.set_name("left_arm_controller")
right_arm_controller = builder.AddSystem(InverseDynamicsController(control_plant, kp, ki, kd, False))
right_arm_controller.set_name("right_arm_controller")

### Decompose Full Plant (22) states to Lower Body (14), Left Arm (4), and Right Arm (4) states 

In [5]:
class DecomposeStates(LeafSystem):
    def __init__(self):
        super().__init__()
        
        # Declare an input port for the full plant state (22 states)
        self.input_port_index = self.DeclareVectorInputPort("full_state", BasicVector(22)).get_index()
        
        # Declare an output port for the lower body states
        self.lower_body_states_output_port_index = self.DeclareVectorOutputPort(
            "lower_body_states", BasicVector(14),  # 14 states in total
            lambda context, output: output.SetFromVector(
                np.hstack([
                    self.get_input_port(self.input_port_index).Eval(context)[:3],         # 1st to 3rd states
                    self.get_input_port(self.input_port_index).Eval(context)[7:14],       # 8th to 14th states
                    self.get_input_port(self.input_port_index).Eval(context)[18:22]      # 19th to 22nd states
                ])
            )
        ).get_index()

        # Declare an output port for the left arm states 
        self.left_arm_states_output_port_index = self.DeclareVectorOutputPort(
            "left_arm_states", BasicVector(4),  # 4 states in total
            lambda context, output: output.SetFromVector(
                np.hstack([
                    self.get_input_port(self.input_port_index).Eval(context)[3:5],         # 4th to 5th states
                    self.get_input_port(self.input_port_index).Eval(context)[14:16],       # 15th to 16th states
                ])
            )
        ).get_index()

        # Declare an output port for the right arm states 
        self.right_arm_states_output_port_index = self.DeclareVectorOutputPort(
            "right_arm_states", BasicVector(4),  # 4 states in total
            lambda context, output: output.SetFromVector(
                np.hstack([
                    self.get_input_port(self.input_port_index).Eval(context)[5:7],         # 6th to 7th states
                    self.get_input_port(self.input_port_index).Eval(context)[16:18],       # 17th to 18th states
                ])
            )
        ).get_index()

### Synthesize Control Input (8) from Lower Body OSC (4), Left Arm PID (2), and Right Arm PID (2)

In [6]:
class CombineControlInputs(LeafSystem):
    def __init__(self):
        super().__init__()
        
        # Declare a single input port that takes a 4-dimensional vector
        self.left_arm_control_input_port_index = self.DeclareVectorInputPort(
            "left_arm_control_input", BasicVector(2)
            ).get_index()
        self.right_arm_control_input_port_index = self.DeclareVectorInputPort(
            "right_arm_control_input", BasicVector(2)
            ).get_index()
        
        # Declare an output port with 8 elements
        self.combined_control_input_output_port_index = self.DeclareVectorOutputPort(
            "combined_control_input", BasicVector(8),
            lambda context, output: output.SetFromVector(
                [
                0, 0, 0, 0, 
                self.get_input_port(self.left_arm_control_input_port_index).Eval(context)[0], 
                self.get_input_port(self.right_arm_control_input_port_index).Eval(context)[0], 
                self.get_input_port(self.left_arm_control_input_port_index).Eval(context)[1], 
                self.get_input_port(self.right_arm_control_input_port_index).Eval(context)[1]
                ]
            )
        ).get_index()

### Calculate Arm Trajectory to follow

In [7]:
# Define a custom system for generating the desired trajectory
class ArmTrajectorySource(LeafSystem):
    def __init__(self):
        super().__init__()
        # Declare an output port for the desired state (position and velocity)
        self.left_arm_traj_output_index = self.DeclareVectorOutputPort(
            "left_arm_traj", BasicVector(4), self.CalcLeftArm).get_index()
        self.right_arm_traj_output_index = self.DeclareVectorOutputPort(
            "right_arm_traj", BasicVector(4), self.CalcRightArm).get_index()

    def CalcLeftArm(self, context, output):
        t = context.get_time()
        theta1 = 1.57 / 10 * np.sin(5*t)          # Target position
        theta1_dot = 1.57 / 10 * np.cos(5*t)      # Target velocity (derivative of sin)
        theta2 = 0          # Target position
        theta2_dot = 0      # Target velocity (derivative of sin)
        output.SetFromVector([theta1, theta2, theta1_dot, theta2_dot])

    def CalcRightArm(self, context, output):
        t = context.get_time()
        theta1 = - 1.57 / 10 * np.sin(5*t)          # Target position
        theta1_dot = - 1.57 / 10 * np.cos(5*t)      # Target velocity (derivative of sin)
        theta2 = 0          # Target position
        theta2_dot = 0      # Target velocity (derivative of sin)
        output.SetFromVector([theta1, theta2, theta1_dot, theta2_dot])

### Decompose States

In [8]:
states_decomposer = DecomposeStates()
builder.AddSystem(states_decomposer)

# wire plant state to state decomposer
builder.Connect(plant.get_state_output_port(), 
                states_decomposer.get_input_port(states_decomposer.input_port_index))

### Connect Arm States to Arm PID Controller

In [9]:
builder.Connect(
    states_decomposer.get_output_port(
        states_decomposer.left_arm_states_output_port_index
        ),  
    left_arm_controller.get_input_port_estimated_state(),
)
builder.Connect(
    states_decomposer.get_output_port(
        states_decomposer.right_arm_states_output_port_index
        ),  
    right_arm_controller.get_input_port_estimated_state(),
)

### Connect Arm Desired Trajectory to Arm PID Controller

In [10]:
# Create and add the trajectory source to generate the desired trajectory
arm_trajectory_source = builder.AddSystem(ArmTrajectorySource())
builder.Connect(
    arm_trajectory_source.get_output_port(
        arm_trajectory_source.left_arm_traj_output_index
        ), 
    left_arm_controller.get_input_port_desired_state()
)
builder.Connect(
    arm_trajectory_source.get_output_port(
        arm_trajectory_source.right_arm_traj_output_index
        ), 
    right_arm_controller.get_input_port_desired_state()
)

### Combine all Control Inputs and wire with Plant Actuation Input

In [11]:
control_input_combiner = CombineControlInputs()
builder.AddSystem(control_input_combiner)

builder.Connect(left_arm_controller.get_output_port(), 
                control_input_combiner.get_input_port(control_input_combiner.left_arm_control_input_port_index))
builder.Connect(right_arm_controller.get_output_port(), 
                control_input_combiner.get_input_port(control_input_combiner.right_arm_control_input_port_index))

builder.Connect(control_input_combiner.get_output_port(control_input_combiner.combined_control_input_output_port_index), 
                plant.get_actuation_input_port())

### Build Diagram

In [12]:
# Build the diagram
diagram = builder.Build()
diagram.set_name("humanoid_walking_controller")

### Run Simulation

In [13]:
# Set initial conditions
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
plant.SetPositions(plant_context, q0)

# Run the simulation
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)

# Start recording the animation
meshcat.StartRecording()
simulator.AdvanceTo(1.5)
meshcat.StopRecording()
meshcat.PublishRecording()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c140ec4a-6224-46c8-b55a-d8a7a25ad3a5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>